In [1]:
import numpy as np
import GCRCatalogs
from astropy.table import Table

from clmm.structures import Cluster

In [2]:
# fyi this is cluster.py

import pickle
from astropy.table import Table

class Cluster():
    def __init__(self, id=None, 
                       ra=None, dec=None,
                       z=None, richness=None,
                       gals=Table()
                ):
        self.id = id
        self.ra = ra
        self.dec = dec
        self.z = z
        self.richness = richness
        self.gals = gals

    def save(self, filename, **kwargs):
        with open(filename, 'wb') as f:
            pickle.dump(self, f, protocol=3, **kwargs)

    def load(self, filename, **kwargs):
        with open(filename, 'rb') as f:
            new_cl = pickle.load(f, **kwargs)
        self.__dict__.update(new_cl.__dict__)

In [3]:
# fyi this is load_from_DC2.py

import os
import numpy as np
import GCRCatalogs
from astropy.table import Table

from clmm.structures import Cluster

def load_from_dc2(N, catalog, save_dir):
    catalog = GCRCatalogs.load_catalog(catalog)
    halos = catalog.get_quantities(['galaxy_id', 'halo_mass', 'redshift','ra', 'dec'],
                                   filters=['halo_mass > 1e14','is_central==True'])

    for _ in range(N):
        i = np.random.randint(len(halos))

        id_cl = halos['galaxy_id'][i]
        ra_cl = halos['ra'][i]
        dec_cl = halos['dec'][i]
        z_cl = halos['redshift'][i]
        m_cl = halos['halo_mass'][i]

        # get galaxies around cluster
        ra_min, ra_max = ra_cl-0.3, ra_cl+0.3
        dec_min, dec_max = dec_cl-0.3, dec_cl+0.3
        z_min = z_cl + 0.1
        z_max = 1.5

        coord_filters = [
            'ra >= %s'%ra_min,
            'ra < %s'%ra_max,
            'dec >= %s'%dec_min,
            'dec < %s'%dec_max,
        ]
        z_filters = ['redshift >= %s'%z_min,
                     'redshift < %s'%z_max]

        gals = catalog.get_quantities(['galaxy_id', 'ra', 'dec', 'shear_1', 'shear_2',
                                       'redshift', 'convergence'], filters=(coord_filters + z_filters))

        # store the results into an astropy table
        t = Table([gals['galaxy_id'],gals['ra'],gals['dec'],gals['shear_1'],
                   gals['shear_2'],gals['redshift'],gals['convergence']],
                  names=('id','ra','dec', 'gamma1', 'gamma2', 'z', 'kappa'))


        c = Cluster(id=id_cl, ra=ra_cl, dec=dec_cl,
                    z=z_cl, richness=None,
                    gals=t)

        c.save(os.path.join(save_dir, '%s.p'%id_cl))



In [4]:
catalog = 'proto-dc2_v2.1.2_test'
save_dir = '/global/homes/m/mho1/scratch/clmm_scratch/cl_from_gcr'

load_from_dc2(5, catalog, save_dir)

/global/common/software/lsst/common/miniconda/py3-4.5.12/envs/stack/lib/python3.7/site-packages/GCRCatalogs/alphaq.py:105: UserWarning: No md5 sum specified in the config file
  warnings.warn('No md5 sum specified in the config file')


In [6]:
load_file = '/global/homes/m/mho1/scratch/clmm_scratch/cl_from_gcr/443618.p'

cl = Cluster()
cl.load(load_file)

print('ra: %s, dec: %s'%(cl.ra, cl.dec))
cl.gals

ra: -0.32084063, dec: -0.15853196


id,ra,dec,gamma1,gamma2,z,kappa
int64,float32,float32,float32,float32,float64,float32
206753,-0.5098095,-0.4126295,0.0028416703,0.011013131,0.47810353929509986,0.0014833808
206983,-0.5147111,-0.31806475,1.0566728e-05,-5.552447e-05,0.4784752016848781,0.0017131567
217473,-0.5866914,-0.34680566,-0.008429415,-0.00041394678,0.5586001394625808,-0.0041689873
217477,-0.5862563,-0.34802443,-0.008245945,-0.00035656933,0.5587265419727918,-0.004394412
217479,-0.42405963,-0.39287058,0.000648918,0.006732483,0.5578912325261296,0.00414896
217480,-0.50749886,-0.40879902,0.003590559,0.011705445,0.558744286089585,0.0018031001
217481,-0.44936177,-0.39948744,0.0042909714,-0.0031751087,0.5580669870667365,0.010678828
217482,-0.5861984,-0.37223566,-0.004256789,0.0021768264,0.5588675533521301,-0.0067566633
217484,-0.51022446,-0.40620226,0.0030498814,0.011795953,0.5587728994011374,0.0013876557
